# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct t-tests and an ANOVA on a real-world dataset and interpret the results

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [2]:
#Your code here
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv("multipleChoiceResponses_cleaned.csv")
df.head()

C:\Users\Vivian\AppData\Local\Temp\ipykernel_5940\3214054283.py:8: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("multipleChoiceResponses_cleaned.csv")


,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,exchangeRate,AdjustedCompensation
0,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,United States,30.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Somewhat important,NaN,NaN,NaN,NaN
2,Male,Canada,28.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,NaN,NaN
3,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",NaN,NaN,Yes,NaN,Operations Research Practitioner,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,250000.0
4,Male,Taiwan,38.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26394 entries, 0 to 26393
Columns: 230 entries, GenderSelect to AdjustedCompensation
dtypes: float64(15), object(215)
memory usage: 46.3+ MB


## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

You may import the functions stored in the `flatiron_stats.py` file to help perform your hypothesis tests. It contains the stats functions that you previously coded: `welch_t(a,b)`, `welch_df(a, b)`, and `p_value(a, b, two_sided=False)`. 

Note that `scipy.stats.ttest_ind(a, b, equal_var=False)` performs a two-sided Welch's t-test and that p-values derived from two-sided tests are two times the p-values derived from one-sided tests. See the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) for more information.    

In [4]:
df.FormalEducation.value_counts()

Master's degree                                                      8204
Bachelor's degree                                                    4811
Doctoral degree                                                      3543
Some college/university study without earning a bachelor's degree     786
Professional degree                                                   451
I did not complete any formal education past high school              257
I prefer not to answer                                                 90
Name: FormalEducation, dtype: int64

Null Hypothesis : There is no difference in salaries.


Alternative Hypothesis: There is a difference in salaries between data scientists with master's degree education and those with bachelor's education.

In [5]:
df1 = df[df["FormalEducation"]== "Master's degree"]
a = df1.AdjustedCompensation
a = a.dropna()
a

3        250000.000
14        36634.400
27        53352.000
31        35419.104
37        80000.000
            ...    
26148     54670.000
26159         1.000
26180     65770.430
26255     89686.950
26378     10000.000
Name: AdjustedCompensation, Length: 1990, dtype: float64

In [6]:
df2 = df[df["FormalEducation"]== "Bachelor's degree"]
b = df2.AdjustedCompensation
b = b.dropna()
b

8         64184.800
9         20882.400
11         1483.900
21        20000.000
25        10858.848
            ...    
26031     39050.000
26072     31878.000
26101      3336.000
26185     50000.000
26195    100449.384
Name: AdjustedCompensation, Length: 1107, dtype: float64

In [7]:
#Your code here
#significance level = 0.05

stats.ttest_ind(a, b, equal_var=False)

Ttest_indResult(statistic=0.43786693335411514, pvalue=0.6615527890254489)

In [8]:
p_value = 0.6615527890254489/2
p_value

0.33077639451272445

In [9]:
a_median = a.median()
print("a_median:", a_median)
b_median = b.median()
print("b_median:", b_median)
a_mean = a.mean()
print("a_mean:", a_mean)
b_mean = b.mean()
print("b_mean:", b_mean)
sample_sizea = len(a)
print("sample_sizea:", sample_sizea)
sample_sizeb = len(b)
print("sample_sizeb:", sample_sizeb)
print("welch t-test pvalue:",p_value)

a_median: 53812.170000000006
b_median: 38399.4
a_mean: 69139.89987120005
b_mean: 64887.09799461879
sample_sizea: 1990
sample_sizeb: 1107
welch t-test pvalue: 0.33077639451272445


Since the pvalue is higher than the significance level (0.05), we fail to reject the null hypothesis.

Therefore, there is no difference in salaries between master's and bachelor's degree holders

## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [10]:
#Your code here

df3 = df[df["FormalEducation"]== "Doctoral degree"]
s1 = df3.AdjustedCompensation
s1 = s1.dropna()
s1.median()

74131.91999999998

In [11]:
df2 = df[df["FormalEducation"]== "Bachelor's degree"]
s2= df2.AdjustedCompensation
s2 = b.dropna()
s2.median()

38399.4

In [12]:
p = stats.ttest_ind(s1, s2, equal_var=False)


In [13]:
p_value = 0.31364763989440514/2
p_value

0.15682381994720257

In [14]:
s1_median = s1.median()
print("s1_median:", s1_median)
s2_median = b.median()
print("s2_median:", s2_median)
s1_mean = s1.mean()
print("s1_mean:", s1_mean)
s2_mean = s2.mean()
print("s2_mean:", s2_mean)
sample_size_s1 = len(s1)
print("sample_size_s1:", sample_size_s1)
sample_size_s2 = len(s2)
print("sample_size_s2:", sample_size_s2)
print("welch t-test pvalue:",p_value)

s1_median: 74131.91999999998
s2_median: 38399.4
s1_mean: 29566175.76245311
s2_mean: 64887.09799461879
sample_size_s1: 967
sample_size_s2: 1107
welch t-test pvalue: 0.15682381994720257


In [15]:
s1.describe()

count    9.670000e+02
mean     2.956618e+07
std      9.099981e+08
min     -7.351631e+01
25%      4.000000e+04
50%      7.413192e+04
75%      1.200000e+05
max      2.829740e+10
Name: AdjustedCompensation, dtype: float64

In [16]:
#finding outliers using interquantile method
Q1 = s1.quantile(0.25)
Q3 = s1.quantile(0.75)
IQR = Q3 - Q1
upper_limit = Q3 + 1.5 * IQR
lower_limit = Q1 - 1.5 * IQR
upper_limit, lower_limit

(240000.0, -80000.0)

In [20]:
new_df = df[(df.AdjustedCompensation > lower_limit)&(df.AdjustedCompensation < upper_limit)]
new_df

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,exchangeRate,AdjustedCompensation
8,Female,Australia,43.0,Employed full-time,NaN,NaN,Yes,NaN,Business Analyst,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.802310,64184.800
9,Male,Russia,33.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017402,20882.400
11,Male,India,27.0,Employed full-time,NaN,NaN,Yes,NaN,Data Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015620,1483.900
14,Male,Taiwan,26.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033304,36634.400
21,Male,United States,25.0,Employed part-time,NaN,NaN,Yes,NaN,Researcher,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,20000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26185,Male,Other,56.0,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,50000.000
26195,Male,Germany,34.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.195826,100449.384
26203,Male,United States,50.0,Employed full-time,NaN,NaN,Yes,NaN,Data Scientist,Perfectly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,200000.000
26255,Male,Norway,32.0,Employed full-time,NaN,NaN,Yes,NaN,Data Analyst,Perfectly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.195826,89686.950


In [21]:
df4 = new_df[new_df["FormalEducation"]== "Doctoral degree"]
new_s1 = df4.AdjustedCompensation
new_s1 = new_s1.dropna()
df5 = new_df[df["FormalEducation"]== "Bachelor's degree"]
new_s2= df5.AdjustedCompensation
new_s2 = new_s2.dropna()

C:\Users\Vivian\AppData\Local\Temp\ipykernel_5940\1643428927.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df5 = new_df[df["FormalEducation"]== "Bachelor's degree"]


In [23]:
new_p = stats.ttest_ind(new_s1, new_s2, equal_var=False)
new_p

Ttest_indResult(statistic=12.381896214999609, pvalue=6.483887013753704e-34)

In [25]:
new_pvalue = 6.483887013753704e-34/2
new_pvalue

3.241943506876852e-34

In [28]:
sample_size = len(new_s1)
print("new_s1 sample size:", sample_size)
sample_sizes = len(new_s2)
print("new_s2 sample size:", sample_sizes)

new_s1 sample size: 934
new_s2 sample size: 1093


## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [82]:
df['FormalEducation'].value_counts()

Master's degree                                                      8204
Bachelor's degree                                                    4811
Doctoral degree                                                      3543
Some college/university study without earning a bachelor's degree     786
Professional degree                                                   451
I did not complete any formal education past high school              257
I prefer not to answer                                                 90
Name: FormalEducation, dtype: int64

In [84]:
#Your code here

import statsmodels.api as sm
from statsmodels.formula.api import ols
a1= 'FormalEducation'
a2= 'AdjustedCompensation'

formula = '{} ~ C({})'.format(a2, a1)
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                          sum_sq      df         F    PR(>F)
C(FormalEducation)  6.540294e+17     6.0  0.590714  0.738044
Residual            7.999414e+20  4335.0       NaN       NaN


## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!